In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import time

In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./AT_interpolated.csv", mode='a', index=False)


In [3]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,
        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./AT_interpolated.csv", mode='a', index=False, header=False)



In [4]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def2/y_pred_{attack_name}{eps}_Def2.npy", all_preds)

        

In [5]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [8]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [10]:
# Adversarial training function with interpolation
def interpolated_adversarial_training(model, train_loader, optimizer, epoch, device, eps=0.3, alpha=2/255, iters=40):
    model.train()
    loss_fn = nn.CrossEntropyLoss()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        # Create adversarial examples
        data_adv = data.clone().detach().requires_grad_(True).to(device)
        outputs = model(data_adv)
        loss = loss_fn(outputs, target)
        loss.backward()
        data_adv = data_adv + alpha * data_adv.grad.sign()
        eta = torch.clamp(data_adv - data, min=-eps, max=eps)
        data_adv = torch.clamp(data + eta, min=0, max=1).detach_()

        # Interpolation
        lam = np.random.beta(1.0, 1.0)
        data_mixed = lam * data + (1 - lam) * data_adv
        target_mixed = lam * target + (1 - lam) * target

        optimizer.zero_grad()
        output = model(data_mixed)
        target_mixed = target_mixed.long()
        
        loss = loss_fn(output, target_mixed)
        loss.backward()
        optimizer.step()

        if batch_idx % 1000 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Model initialization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = input_shape  # Replace with your input shape
output_size = output_shape  # Replace with your output shape
model = DNNModel(input_size=input_size, output_size=output_size).to(device)

# Compile model
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')



In [11]:
# Training loop with early stopping and validation evaluation
for epoch in range(10):
    interpolated_adversarial_training(model, train_loader, optimizer, epoch, device, eps=0.3, alpha=2/255, iters=40)

    # Validation
    model.eval()
    val_train_loss = 0.0
    correct_predictions = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            val_train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct_predictions += (predicted == labels).sum().item()

    avg_val_loss = val_train_loss / len(val_loader)
    val_accuracy = correct_predictions / len(val_dataset)

    print(f"Epoch {epoch+1}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Early stopping check using min_delta
    if best_loss - avg_val_loss > min_delta:
        best_loss = avg_val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break


Train Epoch: 0 [0/82332 (0%)]	Loss: 0.707771
Epoch 1, Validation Loss: 0.1428, Validation Accuracy: 0.9434
Train Epoch: 1 [0/82332 (0%)]	Loss: 0.304557
Epoch 2, Validation Loss: 0.1216, Validation Accuracy: 0.9522
Train Epoch: 2 [0/82332 (0%)]	Loss: 0.098179
Epoch 3, Validation Loss: 0.1200, Validation Accuracy: 0.9513
Train Epoch: 3 [0/82332 (0%)]	Loss: 0.128885
Epoch 4, Validation Loss: 0.1117, Validation Accuracy: 0.9545
Train Epoch: 4 [0/82332 (0%)]	Loss: 0.180925
Epoch 5, Validation Loss: 0.1063, Validation Accuracy: 0.9579
Train Epoch: 5 [0/82332 (0%)]	Loss: 0.153563
Epoch 6, Validation Loss: 0.1464, Validation Accuracy: 0.9403
Train Epoch: 6 [0/82332 (0%)]	Loss: 0.222940
Epoch 7, Validation Loss: 0.1014, Validation Accuracy: 0.9594
Train Epoch: 7 [0/82332 (0%)]	Loss: 0.105542
Epoch 8, Validation Loss: 0.0944, Validation Accuracy: 0.9631
Train Epoch: 8 [0/82332 (0%)]	Loss: 0.081938
Epoch 9, Validation Loss: 0.1025, Validation Accuracy: 0.9601
Train Epoch: 9 [0/82332 (0%)]	Loss: 0

In [11]:
model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Adversarial_Training/Adversarial_Training_interpolated.pt"))


/tmp/ipykernel_2195/413485849.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Adv

<All keys matched successfully>

In [12]:
# calculate_performance_metrics(x_test, y_test, model, 'DNN', 'baseline', '0')

In [13]:
epsilon = 0

Def = "Def2"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                new_row = {
                    "model" : "0",
                    "attack_model" : "0",
                    'epsilon': "0",
                    'Accuracy': "0",
                    'Macro Precision': "0",
                    'Weighted Precision': "0",
                    'Macro Recall': "0",
                    'Weighted Recall': "0",
                    'Macro F1': "0",
                    'Weighted F1': "0",
                }
                new_row_df = pd.DataFrame([new_row])
                new_row_df.to_csv("./AT_interpolated.csv", mode='a', index=False, header=False)



start XGB 100 baseline
(149, 56) (149,)
Accuracy: 0.9261744966442953

macro
Precision: 0.898380355276907
Recall: 0.8900255754475703
F1 Score: 0.8941009239516702

weighted
Precision: 0.9255047582980932
Recall: 0.9261744966442953
F1 Score: 0.9257833555138973

start XGB 100 BIM
(155, 56) (155,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.5290322580645161

macro
Precision: 0.5
Recall: 0.2645161290322581
F1 Score: 0.3459915611814346

weighted
Precision: 1.0
Recall: 0.5290322580645161
F1 Score: 0.6919831223628692

start XGB 100 FGSM
(80, 56) (80,)
Accuracy: 0.975

macro
Precision: 0.5
Recall: 0.4875
F1 Score: 0.4936708860759494

weighted
Precision: 1.0
Recall: 0.975
F1 Score: 0.9873417721518989

start XGB 100 PGD
(155, 56) (155,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.5290322580645161

macro
Precision: 0.5
Recall: 0.2645161290322581
F1 Score: 0.3459915611814346

weighted
Precision: 1.0
Recall: 0.5290322580645161
F1 Score: 0.6919831223628692

start XGB 100 DF
(687, 56) (687,)
Accuracy: 0.9446870451237264

macro
Precision: 0.49693721286370596
Recall: 0.4751098096632504
F1 Score: 0.48577844311377244

weighted
Precision: 0.9880876750681548
Recall: 0.9446870451237264
F1 Score: 0.9659000775741093

start XGB 100 AutoPGD
(365, 56) (365,)
Accuracy: 0.8136986301369863

macro
Precision: 0.7753488372093024
Recall: 0.8700925231307827
F1 Score: 0.7871062924586565

weighted
Precision: 0.8892182223638102
Recall: 0.8136986301369863
F1 Score: 0.8268917278998786

start XGB 100 ZOO
(436, 56) (436,)
Accuracy: 0.9151376146788991

macro
Precision: 0.8807399632642351
Recall: 0.8833215921098978
F1 Score: 0.8820199656269426

weighted
Precision: 0.9154367246912005
Recall: 0.9151376146788991
F1 Score: 0.9152809811250114

start XGB 100 CaFA
(102, 56) (102,)
Accuracy

...

Accuracy: 0.37209302325581395

macro
Precision: 0.5344827586206896
Recall: 0.6707317073170732
F1 Score: 0.3190615835777126

weighted
Precision: 0.9566960705693666
Recall: 0.37209302325581395
F1 Score: 0.49141376253154195

start XGB 1 DF
(49, 56) (49,)
Accuracy: 0.9387755102040817

macro
Precision: 0.5
Recall: 0.46938775510204084
F1 Score: 0.4842105263157895

weighted
Precision: 1.0
Recall: 0.9387755102040817
F1 Score: 0.968421052631579

start XGB 1 AutoPGD
(79, 56) (79,)
Accuracy: 0.4810126582278481

macro
Precision: 0.4813793103448276
Recall: 0.4826923076923077
F1 Score: 0.4725614720729523

weighted
Precision: 0.48139676996944564
Recall: 0.4810126582278481
F1 Score: 0.4717163534574627

start XGB 1 ZOO
(80, 56) (80,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7

macro
Precision: 0.7047619047619047
Recall: 0.701688555347092
F1 Score: 0.6992481203007519

weighted
Precision: 0.7057142857142857
Recall: 0.7
F1 Score: 0.6988721804511279

start XGB 1 CaFA
(5, 56) (5,)
Accuracy: 0.4

macro
Precision: 0.625
Recall: 0.625
F1 Score: 0.4

weighted
Precision: 0.85
Recall: 0.4
F1 Score: 0.4

start XGB 1 SINIFGSM
(31, 56) (31,)
Accuracy: 0.6129032258064516

macro
Precision: 0.6666666666666666
Recall: 0.76
F1 Score: 0.5921052631578947

weighted
Precision: 0.8709677419354839
Recall: 0.6129032258064516
F1 Score: 0.6485568760611206

start XGB 1 VNIFGSM
(12, 56) (12,)
Accuracy: 0.9166666666666666

macro
Precision: 0.8333333333333333
Recall: 0.95
F1 Score: 0.8736842105263158

weighted
Precision: 0.9444444444444445
Recall: 0.9166666666666666
F1 Score: 0.9228070175438595

start RF 100 baseline
(229, 56) (229,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 100 BIM
(2, 56) (2,)

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.9259259259259259

macro
Precision: 0.9308270676691729
Recall: 0.9134199134199135
F1 Score: 0.9205882352941177

weighted
Precision: 0.9271512113617377
Recall: 0.9259259259259259
F1 Score: 0.9251633986928105

start RF 1 CaFA
(1, 56) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 1 SINIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input1/x_test_adv_SINIFGSM_Def2.npy not found
start RF 1 VNIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input1/x_test_adv_VNIFGSM_Def2.npy not found
start DT 100 baseline
(429, 56) (429,)
Accuracy: 0.9393939393939394

macro
Precision: 0.9572541291291292
Recall: 0.8942152103559871
F1 Score: 0.9195655936310142

weighted
Precision: 0.943011193011193
Recall: 0.9393939393939394
F1 Score: 0.9371597595896661

start DT 100 BIM
(3358, 56) (3358,)
Accuracy: 0.3412745681953544

macro
Precision: 0.4585025336

In [18]:
# torch.save(model.state_dict(), "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Adversarial_Training/Adversarial_Training_interpolated.pt")

In [ ]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")